# Distribuindo o TensorFlow por dispositivos e servidores

### Bibliotecas básicas

In [16]:
import numpy as np
import pandas as pd
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
print(tf.__version__)

2.4.0


In [17]:
tf.compat.v1.disable_eager_execution()

### Servidor Local

In [18]:
tf.compat.v1.reset_default_graph()

c = tf.constant("Hello distributed TensorFlow!")
server = tf.compat.v1.train.Server.create_local_server()

with tf.compat.v1.Session(server.target) as sess:
    print(sess.run(c))

b'Hello distributed TensorFlow!'


#### Posicionamento Simples

In [19]:
with tf.compat.v1.device('/cpu:0'):
  a = tf.Variable(3.0)
  b = tf.constant(4.0)
c = a * b

config = tf.compat.v1.ConfigProto()
config.log_device_placement = True

sess = tf.compat.v1.Session(config=config)
a.initializer.run(session=sess)
sess.run(c)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



12.0

#### Posicionamento dinâmico

In [20]:
def variables_on_cpu(op):
  if op.type == 'Variable':
    return '/cpu:0'
  else:
    return '/gpu:0'

with tf.compat.v1.device(variables_on_cpu):
  a = tf.Variable(3.0)
  b = tf.constant(4.0)
c = a * b

config = tf.compat.v1.ConfigProto()
config.log_device_placement = True

sess = tf.compat.v1.Session(config=config)
a.initializer.run(session=sess)
sess.run(c)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



12.0

#### Posicionamento suave

In [21]:
with tf.compat.v1.device('/gpu:0'):
  a = tf.Variable(3)

config = tf.compat.v1.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True

sess = tf.compat.v1.Session(config=config)
sess.run(a.initializer)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



#### Dependência de controle

In [23]:
a = tf.constant(1.0)
b = a + 2.0

with tf.compat.v1.control_dependencies([a, b]):
  x = tf.constant(3.0)
  y = tf.constant(4.0)
z = x + y

### Vários dispositivos em vários servidores

In [27]:
cluster_spec = tf.compat.v1.train.ClusterSpec({
    'ps': [
        '127.0.0.1:2221',
        '127.0.0.1:2222'
    ],
    'worker': [
        '127.0.0.1:2223',
        '127.0.0.1:2224',
        '127.0.0.1:2225'
    ]})


task_ps0 = tf.compat.v1.train.Server(cluster_spec, job_name='ps', task_index=0)
task_ps1 = tf.compat.v1.train.Server(cluster_spec, job_name='ps', task_index=1)

task_worker0 = tf.compat.v1.train.Server(cluster_spec, job_name='worker', task_index=0)
task_worker1 = tf.compat.v1.train.Server(cluster_spec, job_name='worker', task_index=1)
task_worker2 = tf.compat.v1.train.Server(cluster_spec, job_name='worker', task_index=2)

#### Fixar operações em dispositivos e servidores

In [28]:
tf.compat.v1.reset_default_graph()

with tf.compat.v1.device('/job:ps'):
    a = tf.Variable(1.0, name='a')

with tf.compat.v1.device('/job:worker'):
    b = a + 2

with tf.compat.v1.device('/job:worker/task:1'):
    c = a + b

with tf.compat.v1.Session('grpc://127.0.0.1:2221') as sess:
    sess.run(a.initializer)
    print(c.eval())

4.0


#### Particionando as variáveis em múltiplos servidores

In [32]:
tf.compat.v1.reset_default_graph()

with tf.compat.v1.device(tf.compat.v1.train.replica_device_setter(ps_tasks=2, ps_device='/job:ps', worker_device='/job:worker')):
    v1 = tf.Variable(1.0, name='v1')
    v2 = tf.Variable(2.0, name='v2')
    v3 = tf.Variable(3.0, name='v3')
    s = v1 + v2
    with tf.compat.v1.device('/task:1'):
        p1 = 2 * s
        with tf.compat.v1.device('/cpu:0'):
            p2 = 3 * s

config = tf.compat.v1.ConfigProto()
config.log_device_placement = True

with tf.compat.v1.Session('grpc://127.0.0.1:2221', config=config) as sess:
    v1.initializer.run()